# Download stock price data from Yahoo! Finance and load to MySQL

## Load Libraries

In [1]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import MySQLdb
from sqlalchemy import create_engine
from yahoofinancials import YahooFinancials

from pystonks_spd import download_spd

## Metadata

In [2]:
# mysql credentials
hostname = "localhost"
username = "pystonks_dev"
pw = 'pychonkee'

# db metadata
db_name = 'pystonks_dwh'

## Helper Functions

In [3]:
def transform_spd(asset_info, asset_spd):
    
    # transform asset information for db upload
    asset_info = pd.DataFrame(asset_info).transpose()
    drop_cols = \
    ['exchangeTimezoneName', 
     'exchangeTimezoneShortName']
    asset_info = asset_info.drop(columns=drop_cols)

    # transform asset stock price data for db upload
    asset_spd = asset_spd.drop(columns='close')
    asset_spd = asset_spd.rename(columns={'adjclose': 'close'})
    asset_spd['date'] = asset_spd.index
    asset_spd['date'] = pd.to_datetime(asset_spd['date'], format='%Y-%m-%d')
    asset_spd.index = range(len(asset_spd))
    asset_spd['symbol'] = symbol
    
    return asset_info, asset_spd

In [4]:
def execute_query(cursor, query_string, print_out=True):
    
    print('Executing the following query as {} on {}:\n'.format(username, hostname))
    print('mysql>', query_string)
    
    if query_string.count(';') > 1:
        for single_query in query_string.split(';'):
            if len(single_query.strip()) > 0:
                cursor.execute(single_query)
    else:
        cursor.execute(query_string)
        
    if print_out:
        print('\nOutput:\n')
        for line in cursor:
            print(line)
    return


def read_query(filepath):
    with open(filepath, 'r') as handle:
        query_string = handle.read()
    return query_string


def execute_script(cursor, filepath, print_out=True):
    query_string = read_query(filepath)
    execute_query(cursor, query_string, print_out=print_out)
    return

## Download data

In [5]:
%%time

# my watchlist
watch_df = pd.read_csv('data/my_watchlist.csv')
watch_df = watch_df.drop(columns=['remarks'])

watch_info = pd.DataFrame()
watch_spd = pd.DataFrame()
for i, symbol in enumerate(watch_df['symbol']):
    
    print('Downloading data for asset {} out of {}: {}'.format(i+1, len(watch_df), symbol))
    
    # download and transform data
    asset_info, asset_spd = download_spd(symbol)
    asset_info, asset_spd = transform_spd(asset_info, asset_spd)

    # concatenate data for each asset
    watch_info = pd.concat([watch_info, asset_info], ignore_index=True)
    watch_spd = pd.concat([watch_spd, asset_spd], ignore_index=True)

watch_spd['priceId'] = range(len(watch_spd))
display(watch_spd.head(3))
display(watch_spd.dtypes)

print('\nNumber of records per symbol:')
display(pd.DataFrame(watch_spd['symbol'].value_counts()))

,high,low,open,volume,close,date,symbol,priceId
0,96.059998,90.919998,92.110001,51802600,92.300003,2021-01-04,AMD,0
1,93.209999,91.410004,92.099998,34208000,92.769997,2021-01-05,AMD,1
2,92.279999,89.459999,91.620003,51911700,90.330002,2021-01-06,AMD,2


high              float64
low               float64
open              float64
volume              int64
close             float64
date       datetime64[ns]
symbol             object
priceId             int64
dtype: object


Number of records per symbol:


,symbol
AMD,186
PYPL,186
CRWD,186
U,186
MSCI,186
NVDA,186
FICO,186
DDOG,186
SNOW,186
ESTC,186


CPU times: user 2.18 s, sys: 218 ms, total: 2.39 s
Wall time: 2min 9s


## Create pystonks db

In [6]:
# connect to mysql
mydb = MySQLdb.connect(
  host=hostname,
  user=username,
  password=pw
)
mycursor = mydb.cursor()

# execute query to create a database
query_string = "CREATE DATABASE {};".format(db_name)
execute_query(mycursor, query_string)

Executing the following query as pystonks_dev on localhost:

mysql> CREATE DATABASE pystonks_dwh;

Output:



## Create tables

In [7]:
tables_to_create = {
    'watch': watch_df,
    'asset': watch_info,
    'price': watch_spd
}

In [8]:
# connect to database
mydb = MySQLdb.connect(
  host=hostname,
  user=username,
  password=pw,
  database=db_name
)
mycursor = mydb.cursor()

# execute scripts
for key in tables_to_create.keys():
    execute_script(mycursor, 
                   'sql/create_table/create_{}.sql'.format(key),
                   print_out=False)

Executing the following query as pystonks_dev on localhost:

mysql> 

drop table if exists watch;
create table watch(
    symbol          char(10) primary key,
    nickname        char(20),
    watchIndustry   char(20)
);

Executing the following query as pystonks_dev on localhost:

mysql> 

drop table if exists asset;
create table asset(
    symbol          char(10) primary key,
    shortName       char(50),
    market          char(20),
    instrumentType  char(20),
    currency        char(6),
    exchange        char(6),
    firstTradeDate  date
);

Executing the following query as pystonks_dev on localhost:

mysql> 

drop table if exists price;
create table price(
    priceID int primary key,
    symbol  char(10),
    date    date,
    open    decimal(18, 4),
    close   decimal(18, 4),
    high    decimal(18, 4),
    low     decimal(18, 4),
    volume  int
);



## Upload data to db

In [9]:
tables_to_upload = {
    'watch': watch_df,
    'asset': watch_info,
    'price': watch_spd
}

In [10]:
# connect to db
engine_template = 'mysql+mysqlconnector://{0}:{1}@{2}/{3}'
engine = create_engine(engine_template.format(username, 
                                              pw, 
                                              hostname, 
                                              db_name))

# upload tables to db
for table_name in tables_to_upload.keys():
    tables_to_upload[table_name].to_sql(con=engine, 
                                        name=table_name, 
                                        if_exists='append',
                                        index=False)

In [11]:
watch_spd.head()

,high,low,open,volume,close,date,symbol,priceId
0,96.059998,90.919998,92.110001,51802600,92.300003,2021-01-04,AMD,0
1,93.209999,91.410004,92.099998,34208000,92.769997,2021-01-05,AMD,1
2,92.279999,89.459999,91.620003,51911700,90.330002,2021-01-06,AMD,2
3,95.510002,91.199997,91.330002,42897200,95.160004,2021-01-07,AMD,3
4,96.400002,93.269997,95.980003,39816400,94.580002,2021-01-08,AMD,4


## Test Query

In [12]:
execute_script(mycursor, 
               'sql/test_query.sql',
               print_out=False)

Executing the following query as pystonks_dev on localhost:

mysql> 
select
    a.symbol,
    a.currency,
    a.exchange,
    p.priceId,
    p.date,
    p.close
from
    price p
    left join (
        asset a
        ) on p.symbol = a.symbol
where 1=1
    and p.symbol in ('DOCN', 'SNOW', 'DDOG')
    and p.date >= '2021-09-01'
order by date
limit 10;



In [13]:
test_df = pd.read_sql(read_query('sql/test_query.sql'), mydb)

print(test_df.shape)
display(test_df.dtypes)
display(test_df)

(10, 6)


symbol       object
currency     object
exchange     object
priceId       int64
date         object
close       float64
dtype: object

,symbol,currency,exchange,priceId,date,close
0,DOCN,USD,NYQ,856,2021-09-01,63.11
1,DDOG,USD,NMS,1228,2021-09-01,135.77
2,SNOW,USD,NYQ,1042,2021-09-01,301.02
3,DOCN,USD,NYQ,857,2021-09-02,64.15
4,DDOG,USD,NMS,1229,2021-09-02,137.09
5,SNOW,USD,NYQ,1043,2021-09-02,306.89
6,DOCN,USD,NYQ,858,2021-09-03,73.46
7,DDOG,USD,NMS,1230,2021-09-03,138.06
8,SNOW,USD,NYQ,1044,2021-09-03,310.00
9,DOCN,USD,NYQ,859,2021-09-07,70.27
